<a href="https://colab.research.google.com/github/Kitagawa-Mariin/HenryPark_Python/blob/Machine_Learning/5_2_EnsembleLearning_Wine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***Random Forest*** \

Create forest with randomly created decision trees. It creates final prediction from the predictions of each decision trees. They create data randomly from our input samples. \


***Bootstrap Sample***
1. in 1000 samples, get 1 first, put it back to the bag.
2. pick samples with step 1 as same size of input data.

Before training decision trees, pick several features randomly and separate nodes. RandomForestClassifier select squared rooted number of entire feature numbers. So, if it has 4 features, each node uses two features.(For regressor, it uses every features) /

SKlearn train 100 decision trees for random forest. For classification, they get the mean value of probabilities of each classes from every decision trees and the highest value is the prediction. \

For regression, they get the mean value of each predictions.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))
rf.fit(train_input, train_target)
print(rf.feature_importances_) #랜덤 포레스트가 특성의 일부를 램덤하게 선택하여 결정 트리를 훈련하기 때문에, 하나의 특성에 과도하게 집중하지 않고 좀 더 많은 특성이 훈련에 기여할 기회를 얻습니다.
#reduce overfitting,increase generalization.

0.9973541965122431 0.8905151032797809
[0.23167441 0.50039841 0.26792718]


***Out of Bag samples*** \

Remaining samples that are not in bootstrap samples >> evaluate the model with these samples like validate samples.

In [ ]:
rf = RandomForestClassifier(oob_score = True, n_jobs=-1, random_state=42)
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


***Extra Trees*** \

Train 100 decision trees. Similiar with RandomForest. But it does not use bootstrap samples. They use entire training set. BUT, they **RANDOMLY split the node**, not with the best ways like decision tree. \

하나의 결정 트리에서 특성을 무작위로 분할한다면 성능이 낮아지겠지만 많은 트리를 앙상블 하기 때문에 과대적합을 막고 검증 세트의 점수를 높이는 효과가 있다.

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier #ExtraTreesRegressor for regression.
et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [ ]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


***Gradient Boosting*** \

깊이가 얕은 결정 트리를 사용하여 이전 트리의 오차를 보완. 깊이가 3인 결정 트리 100개 사용. 깊이가 얕기 때문에, 과대적합에 강하고 일반적으로 높은 일반화 성능을 기대. \

classifier : logistic log loss \
regressor : 평균 제곱 오차 함수 mean square error \

경사 하강법처럼 가장 낮은 곳을 찾아 천천히 내려감. 깊이가 얕은 트리를 사용하여, 천천히 내려감.

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state = 42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))
#gradient boosting은 결정 트리의 개수를 늘려도 과대적합에 매우 강함.
#학습률(learning rate)을 증가시키고 트리의 개수를 늘리면 조금 더 성능이 향상될 수 있다.


0.8881086892152563 0.8720430147331015


In [ ]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))
#learning_rate의 기본값은 0.1

0.9464595437171814 0.8780082549788999


In [ ]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)
#트리 훈련에 사용할 훈련 세트의 비율을 정하는 subsample : 기본값 1.0 전체 훈련 세트.
#1.0보다 작다면 훈련 세트의 일부.
#그레디언트 부스팅이 랜덤 포레스트보다 조금 더 높은 성능 하지만 순서대로트리를 추가하기 때문에 훈련 속도가 느림.
#GradientBoostingClassifier에는 n_jobs 매개변수가 없다.
#GradientBoostingRegressor


[0.15872278 0.68010884 0.16116839]


***Histrogram-based Gradient Boosting*** \

시간 성능을 높인 gradient boosting \

입력 특성을 256개의 구간으로 나눔(훈련 데이터를 256개 정수구간으로 나눔). 노드를 분할할 때 최적의 분할을 매우 빠르게 찾을 수 있다. \
히스토그램 기반 그레디언트 부스팅은 256개의 구간 중에서 하나를 떼어 놓고 누락된 값을 위해서 사용.\
입력에 누락된 특성이 있더라도 이를 따로 전처리할 필요가 없다. \



In [ ]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))
#n_estimators 대신 부스팅 반복 횟수를 지정하는 max_iter를 사용. 성능을 높이려면 max_iter 매개변수를 테스트.

/usr/local/lib/python3.10/dist-packages/sklearn/experimental/enable_hist_gradient_boosting.py:16: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


0.9321723946453317 0.8801241948619236


permutation_importance(). 특성을 하나씩 랜덤하게 섞어서 모델의 성능이 변화하는지 관차랗여 어떤 특성이 중요한지 계산. \

훈련 세트뿐만 아니라 테스트 세트에도 적용할 수 있고 사이킷런에서 제공하는 추정기 모델에 모두 사용가능. \

n_repeats 매개변수는 랜덤하게 섞을 횟수.


In [ ]:
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats=10, random_state = 42, n_jobs = -1)
print(result.importances_mean)
#importances
#importances_mean
#impotances_std

[0.08876275 0.23438522 0.08027708]


In [ ]:
result = permutation_importance(hgb, test_input, test_target, n_repeats=10, random_state = 42, n_jobs = -1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [ ]:
hgb.score(test_input, test_target)
#HistGradientBoostingRegressor

0.8723076923076923

In [ ]:
#We can use XGBoost class for boosting

from xgboost import XGBClassifier
xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9558403027491312 0.8782000074035686


In [ ]:
#Also LightGBM

from lightgbm import LGBMClassifier
lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)